In [2]:
import cv2
import os
import numpy as np

def enhance_tumor_features(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if image is None:
        raise ValueError("Image not found or unable to read.")

    # Apply Gaussian smoothing
    smoothed_image = cv2.GaussianBlur(image, (5, 5), 0)

    # Convert to Lab color space and apply CLAHE
    lab_image = cv2.cvtColor(smoothed_image, cv2.COLOR_BGR2Lab)
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)
    enhanced_lab = cv2.merge((cl, a_channel, b_channel))
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_Lab2BGR)

    # Edge enhancement techniques
    # Uncomment the one you want to use

    # 1. Laplacian Filter
    # edges_laplacian = cv2.Laplacian(enhanced_image, cv2.CV_64F)
    # edges_laplacian = np.uint8(np.absolute(edges_laplacian))

    # 3. Unsharp Masking
    # blurred = cv2.GaussianBlur(enhanced_image, (5, 5), 0)
    # edges_unsharp = cv2.addWeighted(enhanced_image, 1.5, blurred, -0.5, 0)

    # 4. High-pass Filter
    # kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    # edges_high_pass = cv2.filter2D(enhanced_image, -1, kernel)

    # 5. Bilateral Filter
    edges_bilateral = cv2.bilateralFilter(enhanced_image, d=9, sigmaColor=75, sigmaSpace=75)

    # Select the edge enhancement method to use
    # Uncomment the one you want to use
    # edges = edges_laplacian  # Use Laplacian Filter
    #edges = edges_unsharp  # Use Unsharp Masking
    # edges = edges_high_pass  # Use High-pass Filter
    edges = edges_bilateral   # Use Bilateral Filter

    # Convert single-channel edge image to three channels

    # thresholded_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    # Combine original enhanced image with edges
    final_enhanced_image = cv2.addWeighted(enhanced_image, 0.7, edges, 0.3, 0)

    return final_enhanced_image

def process_images_in_folder(input_folder_path, output_folder_path):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for root, dirs, files in os.walk(input_folder_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
                image_path = os.path.join(root, file)
                relative_path = os.path.relpath(image_path, input_folder_path)
                output_image_path = os.path.join(output_folder_path, relative_path)

                if not os.path.exists(os.path.dirname(output_image_path)):
                    os.makedirs(os.path.dirname(output_image_path))

                if image_path is not None:
                    # Process the image
                    processed_image = enhance_tumor_features(image_path)

                    # Save the processed image to the new folder
                    cv2.imwrite(output_image_path, processed_image)
                else:
                    print(f"Error loading image: {image_path}")

# Example
input_folder_path = "/home/ahmed/PycharmProjects/Lung_Cancer/Final_dataset/train"
output_folder_path = "/home/ahmed/PycharmProjects/Lung_Cancer/Final_dataset/PP"
process_images_in_folder(input_folder_path, output_folder_path)
